# Try CGF! Test run is on a point in time where there's an eddy parked in the beam

In [ ]:
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
import xarray as xr
import ttidelib as tt
import scipy
import cmocean
import os
cmap = cmocean.cm.dense_r
import matplotlib
import matplotlib.pyplot as plt
earth_cmap = matplotlib.cm.get_cmap("gist_earth")
from datetime import timedelta
import filtering
import numpy as np
import dask
from dask.distributed import Client,default_client



def startdask():
    try:
    # Try to get the existing Dask client
        client = default_client()
        print(client)
    except ValueError:
        # If there's no existing client, create a new one
        client = Client()
        print(client)
    return client

client = startdask()


t0 = 44900 # Time in the middle of the window to be filtered
t0 = 38000
time_window = 200 # How many hours either side of t0 to consider
filter_window = 100 # How many hours either side of t0 to actually filter (subset of time window)
zl = slice(0,20)
zl = 5
expt = "notide-20"
filter_cutoff = 2*np.pi/(12.42*3600)
outpath = f"/g/data/nm03/ab8992/postprocessed/{expt}/waves"

## First read in a single that overlaps the eddy

In [ ]:
def cross_scale_transfer(data):

    tau_uu = data.uu - data.u**2
    tau_uv = data.uv - data.u*data.v
    tau_vv = data.vv - data.v**2
    u = data.u
    v = data.v
    uu = data.uu
    vv = data.vv

    transfer = (
        tau_uu * u.differentiate("xb") +
        tau_uv * u.differentiate("yb") +
        tau_uv * v.differentiate("xb") +
        tau_vv * v.differentiate("yb")
    ).rename("energy_transfer")

    return transfer

def plot_transfer(data,vmax = 1.):
    """
    data needs to contain bathy, energy transfer, b"""
    cmap = matplotlib.cm.get_cmap("RdBu")
    fig,ax = plt.subplots(2,figsize = (15,12))
    # Set the background colour to the plot to the lowest value in the cmap
    ax[0].set_facecolor("white")
    ax[1].set_facecolor("white")
    
    if type(vmax) == float or type(vmax) == int:
        data.energy_transfer.plot(ax = ax[0],cmap = cmap,add_colorbar = True,vmax = vmax,vmin = -vmax,cbar_kwargs={'label': "Energy transfer (m^2/s^3)"})
    else:
        data.energy_transfer.plot(ax = ax[0],cmap = cmap,add_colorbar = True,cbar_kwargs={'label': "Energy transfer (m^2/s^3)"})
    data.vorticity.plot(ax = ax[1],cmap = cmap,add_colorbar = True,cbar_kwargs={'label': "Vorticity"},vmax = 0.1,vmin = -0.1)

    data.bathy.plot(cmap = earth_cmap,vmin = -1000,vmax = 1500,ax = ax[0],add_colorbar = False)
    data.bathy.plot(cmap = earth_cmap,vmin = -1000,vmax = 1500,ax = ax[1],add_colorbar = False)

    ax[0].set_xlabel("")
    ax[0].set_title(f"Energy transfer from small to larger scales")
    ax[1].set_title(f"Vorticity")
    plt.show()
    return fig

bathy = xr.open_dataset(f"/g/data/nm03/ab8992/outputs/{expt}/bathy_transect.nc").elevation
bathy = bathy.where(bathy > 0).rename("bathy").drop_vars("lon").drop_vars("lat")

time_window = 200 # How many hours either side of t0 to consider
filter_window = 100 # How many hours either side of t0 to actually filter (subset of time window)




## With tide


In [ ]:
t0 = 44910 # Time in the middle of the window to be filtered
expt = "full-20"
outpath = f"/g/data/nm03/ab8992/postprocessed/{expt}/lfiltered/t0-{t0}"
# rawdata_tide = tt.collect_data(
#     expt,
#     rawdata = ["u","v"],
#     timerange=(t0 - filter_window,t0 + filter_window)).isel(zl = 2,time = 100)


cst = xr.open_mfdataset(outpath + "/*.nc",decode_times = False).isel(time = 100).cst.rename("energy_transfer").persist()
vorticity = tt.calculate_vorticity(rawdata_tide).rename("vorticity")
merged = xr.merge([cst,bathy,vorticity]).load()




In [ ]:
fig = plot_transfer(merged.integrate("zl"),vmax = 1.)


In [ ]:


fig = plot_transfer(merged.integrate("zl"),vmax = 1.)

### Make movie

In [ ]:
t0 = 44900 # Time in the middle of the window to be filtered
expt = "full-20"
outpath = f"/g/data/nm03/ab8992/postprocessed/{expt}/lfiltered/t0-{t0}"
rawdata_tide = tt.collect_data(
    expt,
    rawdata = ["u","v"],
    timerange=(t0 - filter_window,t0 + filter_window)).isel(zl = 2)


cst = xr.open_mfdataset(outpath + "/*.nc",decode_times = False).cst.rename("energy_transfer").persist()
vorticity = tt.calculate_vorticity(rawdata_tide).rename("vorticity")
merged = xr.merge([cst,bathy,vorticity]).load()


tt.make_movie(merged.integrate("zl"),plot_transfer,"full-20","12hr_lagrange_cst",framerate=20)



## Notide

In [ ]:
t0 = 38010
expt = "notide-20"
outpath = f"/g/data/nm03/ab8992/postprocessed/{expt}/lfiltered/t0-{t0}"
# rawdata_tide = tt.collect_data(
#     expt,
#     rawdata = ["u","v"],
#     timerange=(t0 - filter_window,t0 + filter_window)).isel(zl = 2,time = 100)


cst = xr.open_mfdataset(outpath + "/*.nc",decode_times = False).isel(time = 100).cst.rename("energy_transfer").persist()
vorticity = tt.calculate_vorticity(rawdata_tide).rename("vorticity")
merged = xr.merge([cst,bathy,vorticity]).load()




In [ ]:
fig = plot_transfer(merged.integrate("zl"),vmax = 1)


In [ ]:
fig = plot_transfer(merged.integrate("zl"),vmax = 1)


### Make Movie

In [ ]:
t0 = 38000
expt = "notide-20"
outpath = f"/g/data/nm03/ab8992/postprocessed/{expt}/lfiltered/t0-{t0}"
rawdata_tide = tt.collect_data(
    expt,
    rawdata = ["u","v"],
    timerange=(t0 - filter_window,t0 + filter_window)).isel(zl = 2)


cst = xr.open_mfdataset(outpath + "/*.nc",decode_times = False).cst.rename("energy_transfer").persist()
vorticity = tt.calculate_vorticity(rawdata_tide).rename("vorticity")
merged = xr.merge([cst,bathy,vorticity]).load()


tt.make_movie(merged.integrate("zl"),plot_transfer,expt,"12hr_lagrange_cst",framerate=20)

## Plot velocities

In [ ]:
np.log((filtered.u**2)).mean("time").mean("zl").plot()

## Plot outputs of Lagrangian runs